# Explore and clean Dates

parse each date format and extract year to new column
(date:str|datetime) -> year: float
eg.:
- datetime -> year
- '1888.12.21' -> 1888
- '1897.03.1'  -> 1897
- '1901 01.15' -> 1901
- '2017.'      -> 2017
- '1990 k.'    -> 1990
- '1889.10.'   -> 1889
- 1985.0       -> 1985
- empty cell   -> NaN

In [1]:
import pandas as pd
from IPython.display import display, HTML

## Read excel

In [2]:
df = pd.read_excel("./torzskonyv_excel_egyben.xlsx")
df = df[[col for col in df.columns if not col.startswith("Unnamed")]]
df = df[['Összes darab', 'Törzskönyvi szám', 'Megszerzés ideje', 'Leltárcsoport']]

df

,Összes darab,Törzskönyvi szám,Megszerzés ideje,Leltárcsoport
0,48.0,1874/0001,1874.0,"2209-2211 Amerika gyűjtemény, 2212-2215 Törlés..."
1,1456.0,1874/0002,1874.0,"1-10 Ázsia gyűjtemény, 19-84 Ázsia gyűjtemény,..."
2,283.0,1874/0003,1874.0,"11-18 Ázsia gyűjtemény, 158-169 Ázsia gyűjtemé..."
3,45.0,1874/0004,1874.0,"235-240 Ázsia gyűjtemény, 332-334 Ázsia gyűjte..."
4,1.0,1874/0005,1874.0,170 Ázsia gyűjtemény
...,...,...,...,...
11766,59.0,2021/0014,NaN,"2021.20.1-44 Ázsia gyűjtemény, 2021.21.1-2 Eur..."
11767,1.0,2021/0015,2005.0,2021.23.1 Bútor gyűjtemény
11768,2.0,2021/0016,2021-03-05 00:00:00,??? Halászat gyűjtemény
11769,3.0,2021/0017,NaN,??? Bútor gyűjtemény


## Create year column

In [3]:
# normalize dates
import datetime
import math
from typing import Any
import re

def normalize_cell_date(cell)->datetime.datetime:
    if isinstance(cell, datetime.datetime):
        return cell

    elif isinstance(cell, str):
        # handle eg.: '1888.12.21'
        p = re.compile("^(18|19|20)\d\d\.(0[1-9]|1[012])\.\d\d$")
        if p.match(cell):
            # print(cell)
            year, month, day = [int(i) for i in re.split(" |\.", cell)]
            return datetime.datetime(year=year, month=month, day=day)

        # handle eg.: '1897.03.1'
        p = re.compile("^(18|19|20)\d\d\.(0[1-9]|1[012])\.\d$")
        if p.match(cell):
            # print(cell)
            year, month, day = [int(i) for i in cell.split(".")]
            return datetime.datetime(year=year, month=month, day=day)

        # handle eg.: '1901 01.15'
        p = re.compile("^(18|19|20)\d\d\s(0[1-9]|1[012])\.\d\d$")
        if p.match(cell):
            # print(cell)
            year, month, day = [int(i) for i in re.split("\s|\.", cell)]
            return datetime.datetime(year=year, month=month, day=day)

        # handle '2017.'
        p = re.compile("^(18|19|20)\d\d\.$")
        if p.match(cell):
            year = int(cell[:4])
            return datetime.datetime(year=year, month=1, day=1)

        # handle '1990 k.'
        p = re.compile("^(18|19|20)\d\d\sk.$")
        if p.match(cell):
            year = int(cell.split(" ")[0])
            return datetime.date(year=year, month=1, day=1)

        # handle eg.: '1889.10.'
        p = re.compile("^(18|19|20)\d\d.(0[1-9]|1[012]).$")
        if p.match(cell):
            year, month = [int(i) for i in "1889.10.".split(".") if i!=""]
            return datetime.datetime(year=year, month=month, day=1)

    elif isinstance(cell, float) and not math.isnan(cell):
        # handle 1985.0
        return datetime.datetime(year=int(cell), month=1, day=1)

    elif math.isnan(cell):
        # handle NaN
        return cell

    return datetime.datetime.fromisoformat(cell)


def normalize_date_in_row(row):
    row = row.copy()
    cell = row["Megszerzés ideje"]
    try:
        cell = normalize_cell_date(cell)
        if isinstance(cell, datetime.datetime):
            row["Megszerzés ideje"] = cell.year
        elif math.isnan(cell):
            row["Megszerzés ideje"] = float("NaN")
        
    except Exception as err:
        row["Megszerzés ideje"] = float("NaN")
        return row
    return row

df['year'] = df.apply(normalize_date_in_row, axis="columns")["Megszerzés ideje"]
df.to_excel("./torzskonyv_clean_dates.xlsx", index=False)

In [4]:
df = pd.read_excel("./torzskonyv_clean_dates.xlsx")
display("# rest of years")
df[df["year"].isna()]

'# rest of years'

,Összes darab,Törzskönyvi szám,Megszerzés ideje,Leltárcsoport,year
91,NaN,1892/0092,NaN,NaN,NaN
605,65.0,1903/0615,1903.02.30,"41056 ? gyűjtemény, 41081-41084 Közlekedés gyű...",NaN
1269,10.0,1907/1280,1907.02.30,72574-72583 (F8390-F8399) Fényképgyűjtemény,NaN
1270,5.0,1907/1281,1907.02.30,72585-72589 (F15616-F15620) Fényképgyűjtemény,NaN
4283,1.0,1953/0025,1953.02.30,53.25.1 Textil gyűjtemény,NaN
...,...,...,...,...,...
11758,2.0,2021/0006,NaN,2021.4.1.1-2 Egyházi gyűjtemény,NaN
11759,8.0,2021/0007,NaN,2021.8.1-8 Bútor gyűjtemény,NaN
11760,12.0,2021/0008,NaN,2021.5.1-12 Textil gyűjtemény,NaN
11766,59.0,2021/0014,NaN,"2021.20.1-44 Ázsia gyűjtemény, 2021.21.1-2 Eur...",NaN
